In [2]:
import pandas as pd

In [6]:
df = pd.read_csv('../data/data.csv',encoding='utf-8')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
!ls

Untitled.ipynb


In [5]:
!pwd

/home/marcos/Escritorio/AI-prod/ML-Prod-Example/notebooks


In [7]:
# Count the quantity of 'M' (malignant)
count_malignant = df[df['diagnosis'] == 'M'].shape[0]

# Count the quantity of 'B' (benign)
count_benign = df[df['diagnosis'] == 'B'].shape[0]

print("Quantity of 'M' (malignant):", count_malignant)
print("Quantity of 'B' (benign):", count_benign)

Quantity of 'M' (malignant): 212
Quantity of 'B' (benign): 357


In [10]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [11]:
#df = df.drop('Unnamed: 32', axis=1)

In [15]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

def preprocess_breast_cancer(df, test_size=0.2, val_size=0.1, random_state=42):
    # 1. Drop 'Unnamed: 32' if exists
    if 'Unnamed: 32' in df.columns:
        df = df.drop(columns=['Unnamed: 32'])
    
    # 2. Drop rows with null values
    df = df.dropna()

    # 3. Separate features and labels
    # Assuming first column is 'id', second column is 'diagnosis' (labels)
    # Features start from 3rd column onwards
    labels = df['diagnosis']
    features = df.drop(columns=['id', 'diagnosis'])

    # 4. Encode labels: M=1, B=0
    labels_encoded = labels.map({'M': 1, 'B': 0})

    # 5. Split data into train+val and test
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        features, labels_encoded, test_size=test_size, random_state=random_state, stratify=labels_encoded)

    # 6. Split train+val into train and val
    val_relative_size = val_size / (1 - test_size)  # adjust val size relative to train_val split
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=val_relative_size, random_state=random_state, stratify=y_train_val)

    # 7. Convert to TensorFlow datasets
    def df_to_tf_dataset(X, y, batch_size=32, shuffle=True):
        ds = tf.data.Dataset.from_tensor_slices((X.values.astype('float32'), y.values.astype('int32')))
        if shuffle:
            ds = ds.shuffle(buffer_size=len(X))
        ds = ds.batch(batch_size)
        return ds

    train_ds = df_to_tf_dataset(X_train, y_train)
    val_ds = df_to_tf_dataset(X_val, y_val, shuffle=False)
    test_ds = df_to_tf_dataset(X_test, y_test, shuffle=False)

    return train_ds, val_ds, test_ds

# Usage example:
# df = pd.read_csv('breast_cancer_data.csv')
# train_ds, val_ds, test_ds = preprocess_breast_cancer(df)


2025-08-11 15:58:30.296710: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-11 15:58:30.297154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-11 15:58:30.475812: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-11 15:58:30.856062: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-11 15:58:32.906339: W tensorflow/compiler/tf2

In [16]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

def preprocess_data(df):
    if 'Unnamed: 32' in df.columns:
        df = df.drop(columns=['Unnamed: 32'])
    df = df.dropna()
    labels = df['diagnosis'].map({'M': 1, 'B': 0})
    features = df.drop(columns=['id', 'diagnosis'])
    return features, labels

def split_data(features, labels, test_size=0.2, val_size=0.1, random_state=42):
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        features, labels, test_size=test_size, stratify=labels, random_state=random_state)
    val_relative_size = val_size / (1 - test_size)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=val_relative_size, stratify=y_train_val, random_state=random_state)
    return X_train, X_val, X_test, y_train, y_val, y_test

def tf_dataset(X, y, batch_size=32, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((X.values.astype('float32'), y.values.astype('int32')))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X))
    return ds.batch(batch_size)

# Example usage:
# df = pd.read_csv('breast_cancer_data.csv')

features, labels = preprocess_data(df)
X_train, X_val, X_test, y_train, y_val, y_test = split_data(features, labels)
train_ds = tf_dataset(X_train, y_train, shuffle=True)
val_ds = tf_dataset(X_val, y_val, shuffle=False)
test_ds = tf_dataset(X_test, y_test, shuffle=False)


In [17]:
for x,y in train_ds:
    print(x, y)
    break

tf.Tensor(
[[1.546e+01 1.948e+01 1.017e+02 7.489e+02 1.092e-01 1.223e-01 1.466e-01
  8.087e-02 1.931e-01 5.796e-02 4.743e-01 7.859e-01 3.094e+00 4.831e+01
  6.240e-03 1.484e-02 2.813e-02 1.093e-02 1.397e-02 2.461e-03 1.926e+01
  2.600e+01 1.249e+02 1.156e+03 1.546e-01 2.394e-01 3.791e-01 1.514e-01
  2.837e-01 8.019e-02]
 [1.387e+01 2.070e+01 8.977e+01 5.848e+02 9.578e-02 1.018e-01 3.688e-02
  2.369e-02 1.620e-01 6.688e-02 2.720e-01 1.047e+00 2.076e+00 2.312e+01
  6.298e-03 2.172e-02 2.615e-02 9.061e-03 1.490e-02 3.599e-03 1.505e+01
  2.475e+01 9.917e+01 6.886e+02 1.264e-01 2.037e-01 1.377e-01 6.845e-02
  2.249e-01 8.492e-02]
 [9.436e+00 1.832e+01 5.982e+01 2.786e+02 1.009e-01 5.956e-02 2.710e-02
  1.406e-02 1.506e-01 6.959e-02 5.079e-01 1.247e+00 3.267e+00 3.048e+01
  6.836e-03 8.982e-03 2.348e-02 6.565e-03 1.942e-02 2.713e-03 1.202e+01
  2.502e+01 7.579e+01 4.396e+02 1.333e-01 1.049e-01 1.144e-01 5.052e-02
  2.454e-01 8.136e-02]
 [9.742e+00 1.912e+01 6.193e+01 2.897e+02 1.075e-01 8.33

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input




def build_model_with_dropout(dropout_rate=0.1):
    model = Sequential([
        #Input(shape=(None,)),  # None means "any number of features"
        Dense(16, activation='relu'),
        Dropout(dropout_rate),
        Dense(16, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    return model


2025-08-12 15:40:52.941665: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-12 15:40:52.941702: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-12 15:40:52.943017: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-12 15:40:52.949911: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-12 15:40:54.400337: W tensorflow/compiler/tf2

In [15]:
import tensorflow as tf

from tensorflow.keras import metrics

def compile_model(model, learning_rate=1e-3):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[metrics.BinaryAccuracy(name='accuracy'),
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall')]
    )

In [16]:
model = build_model_with_dropout()
compile_model(model, learning_rate=0.005)

In [17]:
model.metrics

[]

In [18]:
import numpy as np

dummy_input = np.random.rand(1, 10).astype('float32')  # batch=1, features=10
_ = model.predict(dummy_input)  # build the model by calling predict

1/1 [==============================] - 0s 90ms/step


In [19]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                176       
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 16)                272       
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 17        
                                                                 
Total params: 465 (1.82 KB)
Trainable params: 465 (1.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
model.metrics

[]

In [ ]:
def test_compile_model():
    model = build_model_with_dropout()
    compile_model(model, learning_rate=0.005)

    assert isinstance(model.optimizer, tf.keras.optimizers.Adam)
    assert abs(model.optimizer.learning_rate.numpy() - 0.005) < 1e-6

    assert model.loss == 'binary_crossentropy'

    #metric_names = [m.name for m in model.metrics] -> da ['loss', 'compile_metroics']

    metric_names = []
    for m in model.metrics:
        # CompileMetrics object contains metric objects
        if hasattr(m, 'metrics'):
            metric_names.extend([metric.name for metric in m.metrics])
        else:
            metric_names.append(m.name)


    print(metric_names)
    assert 'accuracy' in metric_names
    assert 'precision' in metric_names
    assert 'recall' in metric_names

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import metrics
import numpy as np

def build_model_with_dropout(dropout_rate=0.1, input_dim=10):
    model = Sequential([
        Dense(16, activation='relu', input_shape=(input_dim,)),
        Dropout(dropout_rate),
        Dense(16, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    return model

def compile_model(model, learning_rate=1e-3):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            metrics.BinaryAccuracy(name='accuracy'),
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall')
        ]
    )

model = build_model_with_dropout()
compile_model(model)

dummy_x = np.random.rand(1, 10).astype('float32')
model.predict(dummy_x)

print([m.name for m in model.metrics])


1/1 [==============================] - 0s 42ms/step
[]


In [22]:
(print(tf.__version__)) 

2.15.0


In [23]:
import numpy as np

# Build and compile as before
model = build_model_with_dropout()
compile_model(model)

# Dummy input and labels
dummy_x = np.random.rand(4, 10).astype('float32')
dummy_y = np.array([0, 1, 0, 1]).astype('float32')

# Run a single training step (or evaluation)
model.train_on_batch(dummy_x, dummy_y)

# Now metrics are initialized
print([m.name for m in model.metrics])


['loss', 'accuracy', 'precision', 'recall']
